In [1]:
"""
CONNECT WITH DRIVE
"""

%pip install gdown
import gdown
import os
import shutil
from google.colab import drive

# Drive
drive.mount('/content/drive')

# url = 'https://drive.google.com/uc?id=1AvXhv8p9ZCUZ2dzbYoA5CP8VT7a8b6Gf' # full
url = 'https://drive.google.com/uc?id=1Ay6j3rMwSq2qqBSCqC125tFG0c9MEXwW' # train

output = 'dataset_nonrandom_responses.pth'

gdown.download(url, output, quiet=False)

# Github
!git clone https://github.com/karolrogozinski/cern_alice_fast_sim_corrvae.git

source = '/content/cern_alice_fast_sim_corrvae'
destination = '/content'

for file in os.listdir(source):
    source_path = os.path.join(source, file)
    dest_path = os.path.join(destination, file)

    try:
        shutil.copy(source_path, dest_path)
    except IsADirectoryError:
        shutil.copytree(source_path, dest_path)

Mounted at /content/drive


Downloading...
From: https://drive.google.com/uc?id=1Ay6j3rMwSq2qqBSCqC125tFG0c9MEXwW
To: /content/dataset_nonrandom_responses.pth
100%|██████████| 1.84G/1.84G [00:17<00:00, 108MB/s]


Cloning into 'cern_alice_fast_sim_corrvae'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 66 (delta 35), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (66/66), 30.88 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [1]:
"""
CorrVAE training based on [TODO]
"""

import time

import numpy as np
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim

from model import ControlVAE
from encoders import EncoderControlVAE
from decoders import DecoderControlVAE

from utils.helpers import plot_epoch, save_model
from utils.loss import get_losses


In [2]:
"""
PROPERTIES
All of them are normalized

0. - x coordinate of max pixel
1. - y coordinate of max pixel
2. - x coordinate of mass center
3. - y coordinate of mass center
4. - number of non zero pixels
5. - categorized number of non zero pixels
6. - sum of pixels
7. - max pixel value
"""

properties = [2, 3]


In [3]:
"""
ARGUMENTS
"""

device = 'cpu'

results_dir = '/content/drive/MyDrive/models/'
data_source = './data/dataset_nonrandom_responses_train.pth'

img_size = (1, 44, 44)
latent_dim = 8
latent_dim_prop = 8
latent_dim_cond = 9
hid_channels = 32

num_prop = len(properties)
lr = 1e-4

batch_size = 64
epochs = 250

beta = 1
taus = 0.2
idx_kl = 0
w_kl = 100

lambdas = [
    1000000,
    5,
    1000000,
    5,
    1
]


In [5]:
"""
WANDB LOGGER
"""
try:
    import wandb
except:
    %pip install wandb
    import wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [6]:
wandb.login(key='a3dced463ecec1c51f36081c0e372206416cce0a', relogin=True)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(
    project="CorrVAE_64x64",

    config={
        # "data size": train_size,
        "batch_size": batch_size,
        "beta": beta,
        "taus": taus,
        "loss func": 'sigmoid',
        "lr": lr,
        "num prop": num_prop,
        "latent_dim": latent_dim,
        "latent_dim_prop": latent_dim_prop
    }
)


NameError: name 'wandb' is not defined

In [33]:
"""
DATA
"""

data = torch.load(data_source)
dataset = TensorDataset(data['features'], data['labels'])

train_loader = DataLoader(dataset, shuffle=True, batch_size=batch_size)


In [34]:
"""
MODEL
"""

encoder = eval("EncoderControlVAE")
decoder = eval("DecoderControlVAE")

model = ControlVAE(img_size, encoder, decoder, latent_dim, latent_dim_prop, latent_dim_cond, num_prop, device=device)
model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=lr)

lr_scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[50, 100, 150, 200],
        gamma=0.1
)

mse_loss = torch.nn.MSELoss(reduction="sum")

recon_loss_prop_rec = []
recon_loss_rec = []
kl_loss_rec = []
pwwi_loss_rec = []
pwz_loss_rec = []
l1_loss_rec = []
mask_rec = []


In [35]:

latent_dist_cond_mean = data['labels'][:, 9:].mean(axis=0)[0].to(device),
latent_dist_cond_std = data['labels'][:, 9:].std(axis=0)[0].to(device)


In [38]:
latent_dist_cond_mean[0]

tensor([-0.0015, -0.0040,  0.0015,  0.0097,  0.0004,  0.0017, -0.0026, -0.0023,
        -0.0027])

In [36]:
start_epoch = time.time()

for epoch in range(epochs):
    if (epoch + 1) % 10 == 0:
        taus = taus * 0.1

    epoch_loss = []
    epoch_kl_loss = []
    epoch_rec_loss = []
    epoch_prop_loss = []
    epoch_pairwise_loss = []
    epoch_groupwise_loss = []
    epoch_l1_norm = []

    for (data, label) in tqdm(train_loader):
        idx_kl += 1
        data = data.to(device)
        cond = label[:,9:].to(device)

        latent_dist_cond = (
            latent_dist_cond_mean.repeat(data.shape[0], 1),
            latent_dist_cond_std.repeat(data.shape[0], 1)
        )
        latent_sample_cond = label[:, 9:].to(device)

        (reconstruct,y_reconstruct), latent_dist_z, latent_dist_w,\
            latent_sample_z, latent_sample_w, w_mask, cond, mask_ori = model(data, cond, taus)

        latent_sample = torch.cat([latent_sample_w, latent_sample_z], dim=-1)
        latent_dist = (torch.cat([latent_dist_w[0], latent_dist_z[0]], dim=-1),
                       torch.cat([latent_dist_w[1], latent_dist_z[1]], dim=-1))

        ###### Reconstruction loss ######
        rec_loss = F.mse_loss(reconstruct, data, reduction="sum") / 64
        rec_loss = rec_loss / batch_size

        rec_loss_prop = []

        for i, prop in enumerate(properties):
            rec_loss_prop.append(mse_loss(y_reconstruct[:,i], label[:, prop].float().to(device)))

        rec_loss_prop_all = sum(rec_loss_prop)

        ###### Other losses ######
        kl_loss, pairwise_tc_loss, groupwise_tc_loss, l1norm, loss, w_kl = get_losses(
            latent_dist,
            latent_sample_w,
            latent_dist_w,
            beta,
            latent_sample_z,
            latent_dist_z,
            w_mask,
            device,
            idx_kl,
            rec_loss,
            rec_loss_prop_all,
            w_kl,
            len(train_loader.dataset),
            lambdas,
            latent_sample_cond,
            latent_dist_cond
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss.append(float(loss))
        epoch_kl_loss.append(float(kl_loss))
        epoch_rec_loss.append(float(rec_loss))
        epoch_prop_loss.append(float(rec_loss_prop_all))
        epoch_pairwise_loss.append(float(pairwise_tc_loss))
        epoch_groupwise_loss.append(float(groupwise_tc_loss))
        epoch_l1_norm.append(float(l1norm))

    wandb.log(
        {"total_loss ": np.mean(epoch_loss),
            "KL_loss": np.mean(epoch_kl_loss),
            "rec_loss": np.mean(epoch_rec_loss),
            "rec_prop_loss": np.mean(epoch_prop_loss),
            "wwi_loss": np.mean(epoch_pairwise_loss),
            "wz_loss": np.mean(epoch_groupwise_loss),
            "l1_norm": np.mean(epoch_l1_norm),
    })

    save_model(model, optimizer, results_dir, epoch)
    plot_epoch(train_loader, model, device, taus, epoch, time.time() - start_epoch)

    lr_scheduler.step()
    start_epoch = time.time()


  0%|          | 0/3699 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'repeat'